In [8]:
import multiprocessing.popen_spawn_posix
from dask.distributed import Client, progress
import dask.bag as db
import json
import pandas as pd

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:49501 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [3]:
def exception_handler(orig_func):
  def wrapper(*args,**kwargs):
    try:
      return orig_func(*args,**kwargs)
    except:
      print(*args)
  return wrapper
json_except = exception_handler(json.loads)

In [4]:
b = db.read_text('/Users/triznam/Downloads/OpenAccess-GH/metadata/objects/NMAH/*').map(json_except)

In [5]:
nmah_example = b.take(1)[0]
nmah_example

{'id': 'edanmdm-nmah_631116',
 'version': '',
 'unitCode': 'NMAH',
 'linkedId': '0',
 'type': 'edanmdm',
 'content': {'descriptiveNonRepeating': {'record_ID': 'nmah_631116',
   'unit_code': 'NMAH',
   'title_sort': 'THREAD, SKEIN',
   'guid': 'http://n2t.net/ark:/65665/ng49ca746a4-94ea-704b-e053-15f76fa0b4fa',
   'title': {'label': 'Object Name', 'content': 'thread, skein'},
   'metadata_usage': {'access': 'CC0'},
   'data_source': 'National Museum of American History'},
  'indexedStructured': {'object_type': ['Thread, Skein']},
  'freetext': {'setName': [{'label': 'See more items in',
     'content': 'Cultural and Community Life: Textiles'}],
   'identifier': [{'label': 'ID Number', 'content': 'TE.T18808.026'},
    {'label': 'catalog number', 'content': 'T18808.026'},
    {'label': 'catalog number', 'content': 'T.18808.026'}],
   'notes': [{'label': 'Location', 'content': 'Currently not on view'}],
   'physicalDescription': [{'label': 'Physical Description',
     'content': 'artificia

In [6]:
b.count().compute()

1326613

In [15]:
def flatten(record):
    flattened_record = dict()
    if record:
        flattened_record['id'] = record['id']
        flattened_record['unitCode'] = record['unitCode']
        flattened_record['title'] = record['title']
        recordID = record['content'].get('descriptiveNonRepeating', {}).get('record_ID', {})
        flattened_record['record_ID'] = recordID
        metadata = record['content'].get('descriptiveNonRepeating', {}).get('metadata_usage', {})
        flattened_record['metadata_usage'] = metadata
        DataSource = record['content'].get('descriptiveNonRepeating', {}).get('data_source', {})
        flattened_record['data_source'] = DataSource
        ObjectType = record['content'].get('indexedStructured', {}).get('object_type', {})
        flattened_record['object_type'] = ObjectType

        if 'freetext' in record['content']:
            if 'date' in record['content']['freetext']:
                for date in record['content']['freetext']['date']:
                    if date['label'] == 'Date':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'Date made':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'date made':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'associated dates':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'date on object':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'plate date':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'BEP certification date':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'series date':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'alternate calendar date':
                        flattened_record['Date'] = str(date['content'])
                    elif date['label'] == 'patent date':
                        flattened_record['Date'] = str(date['content'])
    return flattened_record

In [16]:
nmah_json = b.map(flatten).compute()
nmah_df = pd.DataFrame(nmah_json)
nmah_df.head()

,id,unitCode,title,record_ID,metadata_usage,data_source,object_type,Date
0,edanmdm-nmah_631116,NMAH,"thread, skein",nmah_631116,{'access': 'CC0'},National Museum of American History,"[Thread, Skein]",NaN
1,edanmdm-nmah_640898,NMAH,"ribbon, jacquard",nmah_640898,{'access': 'CC0'},National Museum of American History,"[Ribbon, Jacquard]",NaN
2,edanmdm-nmah_646671,NMAH,trimming,nmah_646671,{'access': 'CC0'},National Museum of American History,[Trimming],NaN
3,edanmdm-nmah_646782,NMAH,turban square,nmah_646782,{'access': 'CC0'},National Museum of American History,[Turban Square],NaN
4,edanmdm-nmah_630704,NMAH,"fabric, printed",nmah_630704,{'access': 'CC0'},National Museum of American History,"[Fabric, Printed]",NaN


In [17]:
nmah_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326613 entries, 0 to 1326612
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1326357 non-null  object
 1   unitCode        1326357 non-null  object
 2   title           1326357 non-null  object
 3   record_ID       1326357 non-null  object
 4   metadata_usage  1326357 non-null  object
 5   data_source     1326357 non-null  object
 6   object_type     1326357 non-null  object
 7   Date            433101 non-null   object
dtypes: object(8)
memory usage: 81.0+ MB


In [18]:
nmah_df.to_csv('nmah_github_df.tsv.gz', index=False, sep='\t')